In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/acl25_gpt4/without_module_head/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/acl25_gpt4/without_module_head/graph.jsonl'

In [3]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [4]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [5]:
conv[0]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'data_a', 'type': 'input port'}, {'id': 3, 'content': 'data_b', 'type': 'input port'}, {'id': 4, 'content': 'op_code', 'type': 'input port'}, {'id': 5, 'content': 'result', 'type': 'output port'}, {'id': 6, 'content': 'in_latch', 'type': 'submodule'}, {'id': 7, 'content': 'arith_stage', 'type': 'submodule'}, {'id': 8, 'content': 'res_latch', 'type': 'submodule'}, {'id': 9, 'content': 'out_format', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[1, 2, 3, 0, 4, 6, 0, 7, 0, 0, 8, 9], [6, 6, 6, 6, 7, 7, 7, 8, 8, 9, 9, 5]]\n, Design and implement a 5-stage pipeline for a basic arithmetic unit in Verilog.\n\nModule name:  \n    arithmetic_pipeline5               \nInput ports:\n    clk: Clock signal.\n    rst: Reset signal, active high.\n    data_a[31:0]: 32-bit input operand A.\n    data_b[31:0

In [6]:
conv[0]['conversations'][1]['value']

"`define ADD 2'b00\n`define SUB 2'b01\n`define MUL 2'b10\n`define DIV 2'b11\nmodule arithmetic_pipeline5(\n    input clk,\n    input rst,\n    input [31:0] data_a,\n    input [31:0] data_b,\n    input [1:0] op_code,\n    output [31:0] result\n);\nwire [31:0] data_a_latched, data_b_latched, op_result, formatted_result;\n\ninput_latch in_latch (\n    .clk(clk),\n    .rst(rst),\n    .data_a(data_a),\n    .data_b(data_b),\n    .data_a_latched(data_a_latched),\n    .data_b_latched(data_b_latched)\n);\narithmetic_stage arith_stage (\n    .clk(clk),\n    .data_a(data_a_latched),\n    .data_b(data_b_latched),\n    .op_code(op_code),\n    .result(op_result)\n);\nresult_latch res_latch (\n    .clk(clk),\n    .input_result(op_result),\n    .latched_result(formatted_result)\n);\noutput_format out_format (\n    .clk(clk),\n    .input_result(formatted_result),\n    .formatted_result(result)\n);\nendmodule\n\nmodule input_latch(input clk, input rst, input [31:0] data_a, input [31:0] data_b, output re

In [7]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'data_a', 'type': 'input port'}, {'id': 3, 'content': 'data_b', 'type': 'input port'}, {'id': 4, 'content': 'op_code', 'type': 'input port'}, {'id': 5, 'content': 'result', 'type': 'output port'}, {'id': 6, 'content': 'in_latch', 'type': 'submodule'}, {'id': 7, 'content': 'arith_stage', 'type': 'submodule'}, {'id': 8, 'content': 'res_latch', 'type': 'submodule'}, {'id': 9, 'content': 'out_format', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[1, 2, 3, 0, 4, 6, 0, 7, 0, 0, 8, 9], [6, 6, 6, 6, 7, 7, 7, 8, 8, 9, 9, 5]]\n, Design and implement a 5-stage pipeline for a basic arithmetic unit in Verilog.\n\nModule name:  \n    arithmetic_pipeline5               \nInput ports:\n    clk: Clock signal.\n    rst: Reset signal, active high.\n    data_a[31:0]: 32-bit input operand A.\n    data_b[31:0

In [8]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + '\n' + module_head

In [9]:
conv[100]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'instr', 'type': 'input port'}, {'id': 3, 'content': 'opcode', 'type': 'output port'}, {'id': 4, 'content': 'rs1', 'type': 'output port'}, {'id': 5, 'content': 'rs2', 'type': 'output port'}, {'id': 6, 'content': 'rd', 'type': 'output port'}, {'id': 7, 'content': 'imm', 'type': 'output port'}, {'id': 8, 'content': 'func3', 'type': 'output port'}, {'id': 9, 'content': 'func7', 'type': 'output port'}, {'id': 10, 'content': 'decoders', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[2, 2, 10, 10, 10, 10, 10, 10], [10, 3, 4, 5, 6, 7, 8, 9]]\n, Design a modular RISC-V instruction decoder that decodes a 32-bit instruction into its component fields: opcode, register source addresses, destination address, immediate value, and function codes. It uses hierarchical design with submodules for decoding 

In [3]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/acl25_gpt4/with_module_head/conversations.json'



In [ ]:
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)

In [5]:
import json
with open(new_conv_path, 'r') as f:
    new_conv = json.load(f)
    


In [6]:
len(new_conv)

1326

In [ ]:
new_conv[100]['conversations'][0]['value']

In [32]:
len(graph)

2148